In [1]:
"""
Reference: https://www.drivendata.co/blog/predict-pm25-benchmark/

pyhdf appears to be more powerful than gdal, so it may be worth adopting some of the 
methods used here for working with hdf files.

Additionally, the tutorial shows how to make a masked numpy array, which allows us to work
with sparse arrays? (I'm not sure how this works yet.)

Finally, the tutorial explains how to align AOD data with coordinates. This could let us
make some useful model features, like local weather conditions, etc.

"""

import pandas as pd
from datetime import datetime
from osgeo import gdal
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
import keras.backend as backend
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from dateutil import parser
import matplotlib.pyplot as plt
import os



# from pathlib import Path
# import random
# from typing import Dict, List, Union

# from cloudpathlib import S3Path
import geopandas as gpd
# import rasterio

# DATA_PATH = Path.cwd().parent / "data"
# RAW = DATA_PATH / "raw"
# INTERIM = DATA_PATH / "interim"

In [2]:
pm_md = pd.read_csv(
    "pm25_satellite_metadata.csv",
    parse_dates=["time_start", "time_end"],
    index_col=0
)

grid_md = pd.read_csv(
    "grid_metadata.csv",
    index_col=0
)


maiac_md = pm_md[(pm_md["product"] == "maiac") & (pm_md["split"] == "train")].copy()
maiac_md.shape
print(maiac_md['time_start']['20180201T191000_maiac_la_0.hdf'])
pm_md.keys()
print(grid_md)

2018-02-01 17:25:00+00:00
                    location             tz  \
grid_id                                       
1X116                 Taipei    Asia/Taipei   
1Z2W7                  Delhi  Asia/Calcutta   
3S31A    Los Angeles (SoCAB)      Etc/GMT+8   
6EIL6                  Delhi  Asia/Calcutta   
7334C                  Delhi  Asia/Calcutta   
78V83                  Delhi  Asia/Calcutta   
7F1D1                  Delhi  Asia/Calcutta   
8KNI6                  Delhi  Asia/Calcutta   
90BZ1                 Taipei    Asia/Taipei   
90S79                  Delhi  Asia/Calcutta   
9Q6TA                 Taipei    Asia/Taipei   
A2FBI    Los Angeles (SoCAB)      Etc/GMT+8   
A7UCQ                  Delhi  Asia/Calcutta   
AZJ0Z                  Delhi  Asia/Calcutta   
C7PGV                  Delhi  Asia/Calcutta   
CPR0W                  Delhi  Asia/Calcutta   
D72OT                  Delhi  Asia/Calcutta   
D7S1G                  Delhi  Asia/Calcutta   
DHO4M    Los Angeles (SoCAB)      

In [3]:
#Playing around with datasets

from pyhdf.SD import SD, SDC, SDS
import pyproj


hdf = SD('train/20180201T191000_maiac_la_0.hdf')
print(hdf.info())
print(hdf.datasets())


(13, 8)
{'Optical_Depth_047': (('Orbits:grid1km', 'YDim:grid1km', 'XDim:grid1km'), (4, 1200, 1200), 22, 0), 'Optical_Depth_055': (('Orbits:grid1km', 'YDim:grid1km', 'XDim:grid1km'), (4, 1200, 1200), 22, 1), 'AOD_Uncertainty': (('Orbits:grid1km', 'YDim:grid1km', 'XDim:grid1km'), (4, 1200, 1200), 22, 2), 'FineModeFraction': (('Orbits:grid1km', 'YDim:grid1km', 'XDim:grid1km'), (4, 1200, 1200), 22, 3), 'Column_WV': (('Orbits:grid1km', 'YDim:grid1km', 'XDim:grid1km'), (4, 1200, 1200), 22, 4), 'AOD_QA': (('Orbits:grid1km', 'YDim:grid1km', 'XDim:grid1km'), (4, 1200, 1200), 23, 5), 'AOD_MODEL': (('Orbits:grid1km', 'YDim:grid1km', 'XDim:grid1km'), (4, 1200, 1200), 21, 6), 'Injection_Height': (('Orbits:grid1km', 'YDim:grid1km', 'XDim:grid1km'), (4, 1200, 1200), 5, 7), 'cosSZA': (('Orbits:grid5km', 'YDim:grid5km', 'XDim:grid5km'), (4, 240, 240), 22, 8), 'cosVZA': (('Orbits:grid5km', 'YDim:grid5km', 'XDim:grid5km'), (4, 240, 240), 22, 9), 'RelAZ': (('Orbits:grid5km', 'YDim:grid5km', 'XDim:grid5km'

In [4]:
blue_band_AOD = hdf.select("Optical_Depth_047")
name, num_dim, shape, types, num_attr = blue_band_AOD.info()
print(
f"""Dataset name: {name}
Number of dimensions: {num_dim}
Shape: {shape}
Data type: {types}
Number of attributes: {num_attr}"""
)

Dataset name: Optical_Depth_047
Number of dimensions: 3
Shape: [4, 1200, 1200]
Data type: 22
Number of attributes: 6


In [5]:
blue_band_AOD.get()

array([[[-28672, -28672, -28672, ..., -28672, -28672, -28672],
        [-28672, -28672, -28672, ..., -28672, -28672, -28672],
        [-28672, -28672, -28672, ..., -28672, -28672, -28672],
        ...,
        [-28672, -28672, -28672, ..., -28672, -28672, -28672],
        [-28672, -28672, -28672, ..., -28672, -28672, -28672],
        [-28672, -28672, -28672, ..., -28672, -28672, -28672]],

       [[-28672, -28672, -28672, ..., -28672, -28672, -28672],
        [-28672, -28672, -28672, ..., -28672, -28672, -28672],
        [-28672, -28672, -28672, ..., -28672, -28672, -28672],
        ...,
        [    21,     21,     21, ..., -28672, -28672, -28672],
        [    22,     21,     21, ..., -28672, -28672, -28672],
        [    24,     22,     21, ..., -28672, -28672, -28672]],

       [[-28672, -28672, -28672, ..., -28672, -28672, -28672],
        [-28672, -28672, -28672, ..., -28672, -28672, -28672],
        [-28672, -28672, -28672, ..., -28672, -28672, -28672],
        ...,
        [   

In [6]:
calibration_dict = blue_band_AOD.attributes()
calibration_dict

{'long_name': 'AOD at 0.47 micron',
 'scale_factor': 0.001,
 'add_offset': 0.0,
 'unit': 'none',
 '_FillValue': -28672,
 'valid_range': [-100, 5000]}

In [7]:
raw_attr = hdf.attributes()["StructMetadata.0"]
group_1 = raw_attr.split("END_GROUP=GRID_1")[0]
# print(group_1)

hdf_metadata = dict([x.split("=") for x in group_1.split() if "=" in x])

# Parse expressions still wrapped in apostrophes
for key, val in hdf_metadata.items():
    try:
        hdf_metadata[key] = eval(val)
    except:
        pass

hdf_metadata

{'GROUP': 'MergedFields',
 'END_GROUP': 'MergedFields',
 'GridName': 'grid1km',
 'XDim': 1200,
 'YDim': 1200,
 'UpperLeftPointMtrs': (-11119505.196667, 4447802.078667),
 'LowerRightMtrs': (-10007554.677, 3335851.559),
 'Projection': 'GCTP_SNSOID',
 'ProjParams': (6371007.181, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 'SphereCode': -1,
 'GridOrigin': 'HDFE_GD_UL',
 'OBJECT': 'DataField_8',
 'DimensionName': 'Orbits',
 'Size': 4,
 'END_OBJECT': 'DataField_8',
 'DataFieldName': 'Injection_Height',
 'DataType': 'DFNT_FLOAT32',
 'DimList': ('Orbits', 'YDim', 'XDim')}

In [8]:
# Note that coordinates are provided in meters
alignment_dict = {
    "upper_left": hdf_metadata["UpperLeftPointMtrs"],
    "lower_right": hdf_metadata["LowerRightMtrs"],
    "crs": hdf_metadata["Projection"],
    "crs_params": hdf_metadata["ProjParams"]
}
alignment_dict

{'upper_left': (-11119505.196667, 4447802.078667),
 'lower_right': (-10007554.677, 3335851.559),
 'crs': 'GCTP_SNSOID',
 'crs_params': (6371007.181, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)}

In [9]:
"""
DATA PROCESSING

"""

'\nDATA PROCESSING\n\n'

In [10]:
# Loop over orbits to apply the attributes
def calibrate_data(dataset: SDS, shape: list[int], calibration_dict: dict):
    """Given a MAIAC dataset and calibration parameters, return a masked
    array of calibrated data.
    
    Args:
        dataset (SDS): dataset in SDS format (e.g. blue band AOD).
        shape (List[int]): dataset shape as a list of [orbits, height, width].
        calibration_dict (Dict): dictionary containing, at a minimum,
            `valid_range` (list or tuple), `_FillValue` (int or float),
            `add_offset` (float), and `scale_factor` (float).
    
    Returns:
        corrected_AOD (np.ma.MaskedArray): masked array of calibrated data
            with a fill value of nan.
    """
    corrected_AOD = np.ma.empty(shape, dtype=np.double)
    for orbit in range(shape[0]):
        data = dataset[orbit, :, :].astype(np.double)
        invalid_condition = (
            (data < calibration_dict["valid_range"][0]) |
            (data > calibration_dict["valid_range"][1]) |
            (data == calibration_dict["_FillValue"])
        )
        data[invalid_condition] = np.nan
        data = (
            (data - calibration_dict["add_offset"]) *
            calibration_dict["scale_factor"]
        )
        data = np.ma.masked_array(data, np.isnan(data))
        corrected_AOD[orbit, : :] = data
    corrected_AOD.fill_value = np.nan
    return corrected_AOD



In [11]:
corrected_AOD = calibrate_data(blue_band_AOD, shape, calibration_dict)
corrected_AOD

masked_array(
  data=[[[--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         ...,
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --]],

        [[--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         ...,
         [0.021, 0.021, 0.021, ..., --, --, --],
         [0.022, 0.021, 0.021, ..., --, --, --],
         [0.024, 0.022, 0.021, ..., --, --, --]],

        [[--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         ...,
         [0.073, 0.065, 0.053, ..., --, --, --],
         [0.062, 0.057, 0.051000000000000004, ..., --, --, --],
         [0.056, 0.048, 0.041, ..., --, --, --]],

        [[--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         ...,
         [--, -

In [12]:
pd.DataFrame(corrected_AOD.ravel(), columns=['AOD']).describe()

,AOD
count,1.203568e+06
mean,9.204344e-02
std,6.629422e-02
min,0.000000e+00
25%,4.500000e-02
50%,7.200000e-02
75%,1.220000e-01
max,5.940000e-01


In [13]:
"""

Aligning AOD data with real world coordinates


"""


def create_meshgrid(alignment_dict: dict, shape: list[int]):
    """Given an image shape, create a meshgrid of points
    between bounding coordinates.
    
    Args:
        alignment_dict (Dict): dictionary containing, at a minimum,
            `upper_left` (tuple), `lower_right` (tuple), `crs` (str),
            and `crs_params` (tuple).
        shape (List[int]): dataset shape as a list of
            [orbits, height, width].
    
    Returns:
        xv (np.array): x (longitude) coordinates.
        yv (np.array): y (latitude) coordinates.
    """
    # Determine grid bounds using two coordinates
    x0, y0 = alignment_dict["upper_left"]
    x1, y1 = alignment_dict["lower_right"]
    
    # Interpolate points between corners, inclusive of bounds
    x = np.linspace(x0, x1, shape[2], endpoint=True)
    y = np.linspace(y0, y1, shape[1], endpoint=True)
    
    # Return two 2D arrays representing X & Y coordinates of all points
    xv, yv = np.meshgrid(x, y)
    return xv, yv

xv, yv = create_meshgrid(alignment_dict, shape)

In [14]:
from pyproj import CRS, Proj
from typing import Union

# Source: https://spatialreference.org/ref/sr-org/modis-sinusoidal/proj4js/
sinu_crs = Proj(f"+proj=sinu +R={alignment_dict['crs_params'][0]} +nadgrids=@null +wktext").crs
wgs84_crs = CRS.from_epsg("4326")

def transform_arrays(
    xv: Union[np.array, float],
    yv: Union[np.array, float],
    crs_from: CRS,
    crs_to: CRS
):
    """Transform points or arrays from one CRS to another CRS.
    
    Args:
        xv (np.array or float): x (longitude) coordinates or value.
        yv (np.array or float): y (latitude) coordinates or value.
        crs_from (CRS): source coordinate reference system.
        crs_to (CRS): destination coordinate reference system.
    
    Returns:
        lon, lat (tuple): x coordinate(s), y coordinate(s)
    """
    transformer = pyproj.Transformer.from_crs(
        crs_from,
        crs_to,
        always_xy=True,
    )
    lon, lat = transformer.transform(xv, yv)
    return lon, lat



# Project sinu grid onto wgs84 grid
lon, lat = transform_arrays(xv, yv, sinu_crs, wgs84_crs)

In [15]:
def convert_array_to_df(
    corrected_arr: np.ma.MaskedArray,
    lat:np.ndarray,
    lon: np.ndarray,
    granule_id: str,
    crs: CRS,
    total_bounds: np.ndarray = None
):
    """Align data values with latitude and longitude coordinates
    and return a GeoDataFrame.
    
    Args:
        corrected_arr (np.ma.MaskedArray): data values for each pixel.
        lat (np.ndarray): latitude for each pixel.
        lon (np.ndarray): longitude for each pixel.
        granule_id (str): granule name.
        crs (CRS): coordinate reference system
        total_bounds (np.ndarray, optional): If provided,
            will filter out points that fall outside of these bounds.
            Composed of xmin, ymin, xmax, ymax.
    """
    lats = lat.ravel()
    lons = lon.ravel()
    n_orbits = len(corrected_arr)
    size = lats.size
    values = {
        "value": np.concatenate([d.data.ravel() for d in corrected_arr]),
        "lat": np.tile(lats, n_orbits),
        "lon": np.tile(lons, n_orbits),
        "orbit": np.arange(n_orbits).repeat(size),
        "granule_id": [granule_id] * size * n_orbits
        
    }
    
    df = pd.DataFrame(values).dropna()
    if total_bounds is not None:
        x_min, y_min, x_max, y_max = total_bounds
        df = df[df.lon.between(x_min, x_max) & df.lat.between(y_min, y_max)]
    
    gdf = gpd.GeoDataFrame(df)
    gdf["geometry"] = gpd.points_from_xy(gdf.lon, gdf.lat)
    gdf.crs = crs
    return gdf[["granule_id", "orbit", "geometry", "value"]].reset_index(drop=True)

gdf = convert_array_to_df(corrected_AOD, lat, lon, '20180201T191000_maiac_la_0.hdf', wgs84_crs)
print(gdf.shape)
gdf.head(3)

(1203568, 4)


,granule_id,orbit,geometry,value
0,20180201T191000_maiac_la_0.hdf,0,POINT (-110.79078 35.36280),0.110
1,20180201T191000_maiac_la_0.hdf,0,POINT (-110.78956 35.35446),0.076
2,20180201T191000_maiac_la_0.hdf,0,POINT (-110.28947 35.28774),0.112


In [16]:
"""

Some more helpful functions from the tutorial


"""

def create_calibration_dict(data: SDS):
    """Define calibration dictionary given a SDS dataset,
    which contains:
        - name
        - scale factor
        - offset
        - unit
        - fill value
        - valid range
    
    Args:
        data (SDS): dataset in the SDS format.
    
    Returns:
        calibration_dict (Dict): dict of calibration parameters.
    """
    return data.attributes()


def create_alignment_dict(hdf: SD):
    """Define alignment dictionary given a SD data file, 
    which contains:
        - upper left coordinates
        - lower right coordinates
        - coordinate reference system (CRS)
        - CRS parameters
    
    Args:
        hdf (SD): hdf data object
    
    Returns:
        alignment_dict (Dict): dict of alignment parameters.
    """
    group_1 = hdf.attributes()["StructMetadata.0"].split("END_GROUP=GRID_1")[0]
    hdf_metadata = dict([x.split("=") for x in group_1.split() if "=" in x])
    alignment_dict = {
        "upper_left": eval(hdf_metadata["UpperLeftPointMtrs"]),
        "lower_right": eval(hdf_metadata["LowerRightMtrs"]),
        "crs": hdf_metadata["Projection"],
        "crs_params": hdf_metadata["ProjParams"]
    }
    return alignment_dict

In [21]:
"""

Importing HDF data and creating a set of raw hdf files.


"""

train_labels = pd.read_csv("train_labels.csv") # Smallest subset
grid_metadata = pd.read_csv("grid_metadata.csv")
satellite_metadata = pd.read_csv("pm25_satellite_metadata.csv")
satellite_metadata = satellite_metadata[satellite_metadata.granule_id.str.endswith('f')]
satellite_metadata = satellite_metadata[satellite_metadata['split'] == 'train']
print(satellite_metadata)
raw_hdf_set = set(satellite_metadata['granule_id'])


                          granule_id                time_start  \
0     20180201T191000_maiac_la_0.hdf  2018-02-01T17:25:00.000Z   
1     20180202T195000_maiac_la_0.hdf  2018-02-02T18:05:00.000Z   
2     20180203T203000_maiac_la_0.hdf  2018-02-03T17:10:00.000Z   
3     20180204T194000_maiac_la_0.hdf  2018-02-04T17:55:00.000Z   
4     20180205T202000_maiac_la_0.hdf  2018-02-05T17:00:00.000Z   
...                              ...                       ...   
4255  20201227T071500_maiac_dl_0.hdf  2020-12-27T05:25:00.000Z   
4256  20201228T062000_maiac_dl_0.hdf  2020-12-28T06:10:00.000Z   
4257  20201229T070000_maiac_dl_0.hdf  2020-12-29T05:15:00.000Z   
4258  20201230T060500_maiac_dl_0.hdf  2020-12-30T05:55:00.000Z   
4259  20201231T065000_maiac_dl_0.hdf  2020-12-31T05:00:00.000Z   

                       time_end product location  split  \
0     2018-02-01 19:10:00+00:00   maiac       la  train   
1     2018-02-02 19:50:00+00:00   maiac       la  train   
2     2018-02-03 20:30:00+00:0

In [37]:
from shapely.geometry import Point, Polygon

"""

Everything here is original code that uses the functions from the tutorial.

within(): taken from https://automating-gis-processes.github.io/2017/lessons/L3/point-in-polygon.html

Make_Submatrix(): A function which takes raw AOD matrix and a Grid ID of interest as input and outputs a submatrix 
of AOD values which are inside this grid point (5km by 5km).

Currently, Make_Submatrix() only returns the number of pixels in the AOD matrix are within the location determined
by Grid ID.

The rest of the code in this cell runs extremely slowly, but this is because we are running it for all possible 
combinations of HDF file and Grid ID. When we actually use these functions to run a model on a given Grid ID and 
datetime, we will first filter the set of HDF files such that we only search through HDF files with matching city 
and matching datetime.

"""

#Helper function
def Make_Poly(polyString):
    poly_coords = []
    for string in polyString.split(','):
        split_string = string.split(' ')
        if split_string[0] == 'POLYGON':
            split_string = split_string[1:]
            split_string[0] = str(split_string[0])[2:]
    #         print(tuple(float(x) for x in split_string))
        elif split_string[0] == '':
            split_string = split_string[1:]
        if split_string[1][-2] == ')':
            split_string[1] = split_string[1][0:-2]
        poly_coords.append(tuple(float(x) for x in split_string))

    return Polygon(poly_coords)


#Main function
def Make_Submatrix(corrected_AOD, lon, lat, alignment_dict, grid_md, gridID):
#     xv, yv = create_meshgrid(alignment_dict, shape)
#     lon, lat = transform_arrays(xv, yv, sinu_crs, wgs84_crs)
    
    poly = Make_Poly(grid_md['wkt'][gridID])
    counter = 0
    for i in range(len(corrected_AOD[0])):
        for j in range(len(corrected_AOD[0][0])):
            p1 = Point(lon[i,j], lat[i,j]) 
#             print(p1, poly)
            if(p1.within(poly)):
                counter+=1
    
    if counter > 0:
        print(counter)
            

    
    #     print(tuple(float(x) for x in split_string))
    
    
for hdf_filename in raw_hdf_set:
    print(hdf_filename)
    filepath = 'train/' + hdf_filename
    raw_hdf = SD(filepath)
    
    alignment_dict = create_alignment_dict(raw_hdf)
#     print(alignment_dict['upper_left'], alignment_dict['lower_right'])
    
    blue_band_AOD = raw_hdf.select("Optical_Depth_047")
    name, num_dim, shape, types, num_attr = blue_band_AOD.info()
    calibration_dict = create_calibration_dict(blue_band_AOD)
    corrected_AOD = calibrate_data(blue_band_AOD, shape, calibration_dict)
    
    xv, yv = create_meshgrid(alignment_dict, shape)
    lon, lat = transform_arrays(xv, yv, sinu_crs, wgs84_crs)
    
    
    for gridID in grid_md['wkt'].keys():
        Make_Submatrix(corrected_AOD, lon, lat, alignment_dict, grid_md, gridID)
    

20180810T061500_maiac_dl_0.hdf
23
24
23
24
24


KeyboardInterrupt: 

In [35]:
print(len(corrected_AOD[0][0]))

1200
